In [ ]:
TRUST_REGION_COLOR = 'gold'
OPTIMAL_VERTEX_COLOR = 'limegreen'
DROP_VERTEX_COLOR = 'red'
OTHER_VERTEX_COLOR = 'blue'
TRACK_COLOR = 'darkred'

# Delta branch

In [314]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.tri as tri

def ax_plot_simplex(ax, optimal_vertex, sim, jdrop, rho):
    simplex = np.array((optimal_vertex, *(sim + optimal_vertex), optimal_vertex))
    ax.plot(simplex[...,0], simplex[...,1], '-', color='lightsteelblue', lw=2)
    ax.plot(simplex[1:-1,0], simplex[1:-1,1], '.', color='blue', marker='o')
    ax.plot(simplex[jdrop+1,0], simplex[jdrop+1,1], '.', color='red', marker='o')
    ax.plot(*optimal_vertex, color='limegreen', marker='D')
    
    beta = 2.1
    beta_region = plt.Circle(optimal_vertex, beta * rho, color='lemonchiffon', fill=True, alpha=0.5)
    trust_region = plt.Circle(optimal_vertex, rho, color='gold', fill=True, alpha=0.5)
    ax.add_patch(beta_region)
    ax.add_patch(trust_region)

In [324]:
# eta case
plt.close('all')
jdrop = 0
rho = 0.0625
optimal_vertex = np.array([0.047873700624370373047, 0.03301727499567987112])
old_sim = np.array([[-0.13019926157022278929, -0.059595187823573939068], [-0.05055567780160368646, 0.036747427692566961288]])
new_sim = np.array([[0.018373713846283478442, 0.025277838900801840201], [-0.05055567780160368646, 0.036747427692566961288]])
old_eta = [0.14319020261581568261, 0.062500000000000007495]
new_eta = [0.03125, 0.062500000000000007495]
old_vsig = [0.12475780902899252224, 0.054454585033534948986]
new_vsig = [0.031249999999999999998, 0.06250000000000000749]

coords = np.array((*old_sim, *new_sim)) + optimal_vertex
x_coords = coords[:, 0]
y_coords = coords[:, 1]

plt.close('all')
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))
xmin = min(x_coords)
xmax = max(x_coords)
ymin = min(y_coords)
ymax = max(y_coords)
margin = 1e-2

#_ = ax1.set(xlim=(xmin - margin, xmax + margin), ylim=(ymin - margin, ymax + margin))
#_ = ax2.set(xlim=(xmin - margin, xmax + margin), ylim=(ymin - margin, ymax + margin))
ax1.set_aspect('equal')
ax2.set_aspect('equal')

ax_plot_simplex(ax1, optimal_vertex, old_sim, jdrop, rho=rho)
ax_plot_simplex(ax2, optimal_vertex, new_sim, jdrop, rho=rho)
plt.savefig('mem/rsc/delta-eta.png', dpi=120, bbox_inches='tight')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [323]:
# sigma case
plt.close('all')
jdrop = 1
rho = 6.103515625e-05
optimal_vertex = np.array([0.12322095602324728612, 0.7876477600799215061])
old_sim = np.array([[0.00010609379901699572403, -5.758107655831259315e-05], [4.8716293255678108707e-05, -3.676973034832708352e-05]])
new_sim = np.array([[0.00010609379901699572403, -5.758107655831259315e-05], [-1.4557208222393858973e-05, -2.6821824385849429767e-05]])
old_veta = [0.00012071236294378856573, 6.103515625000000053e-05]
new_eta = [0.00012071236294378856573, 3.0517578125e-05]
old_vsig = [1.7955287365912709351e-05, 9.078637375381810534e-06]
new_vsig = [0.00012071236294378856568, 3.0517578124999999998e-05]

coords = np.array((*old_sim, *new_sim)) + optimal_vertex
x_coords = coords[:, 0]
y_coords = coords[:, 1]

plt.close('all')
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))
xmin = min(x_coords)
xmax = max(x_coords)
ymin = min(y_coords)
ymax = max(y_coords)
margin = 6e-5

#_ = ax1.set(xlim=(xmin - margin, xmax + margin), ylim=(ymin - margin, ymax + margin))
#_ = ax2.set(xlim=(xmin - margin, xmax + margin), ylim=(ymin - margin, ymax + margin))
ax1.set_aspect('equal')
ax2.set_aspect('equal')

ax_plot_simplex(ax1, optimal_vertex, old_sim, jdrop, rho=rho)
ax_plot_simplex(ax2, optimal_vertex, new_sim, jdrop, rho=rho)
plt.savefig('mem/rsc/delta-sigma.png', dpi=120, bbox_inches='tight')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Examples

In [338]:
import functools
import itertools

import numpy as np

from pycobyla.gcobyla import GCobyla

import tests.test_custom as tc
from tests.nb import utils

In [415]:
%matplotlib widget

import matplotlib.pyplot as plt
from matplotlib import cm

def ax_plot_simplex(ax, optimal_vertex, sim):
    simplex = np.array((optimal_vertex, *(sim + optimal_vertex), optimal_vertex))
    ax.plot(*optimal_vertex, color='limegreen', marker='o')
    ax.plot(simplex[1:-1,0], simplex[1:-1,1], '.', color='blue', marker='o')
    ax.plot(simplex[...,0], simplex[...,1], '-.', color='gray', lw=2)
    

def ax_plot_track(opt, ax, target, from_points=0, to_points=-1, plot_simplex=False, plot_trure=False, aspect=False, track_color=None, track_alpha=1):
    track = opt.track[from_points:to_points]
    ax.plot(track[:, 0], track[:, 1], linestyle='-', color=track_color, alpha=track_alpha, marker='o')

    link_best = np.array((track[-1], target))
    ax.plot(link_best[:, 0], link_best[:, 1], linestyle=':', color=track_color)
    ax.plot(target[0], target[1], color='limegreen', marker='*')
        
    if plot_simplex:
        #simplex = np.array((opt.optimal_vertex, *(opt.sim + opt.optimal_vertex), opt.optimal_vertex))
        ax_plot_simplex(ax, opt.optimal_vertex, opt.sim)
            
    if plot_trure:
        trust_region = plt.Circle(opt.optimal_vertex, opt.rho, color='khaki', fill=True, alpha=0.5)
        ax.add_patch(trust_region)
        
    if aspect:
        ax.set_aspect('equal')
        
def ax_plot_map(fig, ax, opt, border, nsamples=100):
    X = np.linspace(-border, border, nsamples)
    xv, yv = np.meshgrid(X, X)
    zv = np.array(tuple(-opt.F(xy) for xy in itertools.product(X, X)))
    zv = zv.reshape(xv.shape)
    
    cs = ax.contourf(xv, yv, zv, cmap=cm.magma)
    #cbar = fig.colorbar(cs)
    
    #ax.plot(np.array((-border, border)), np.array((0, 0)), linestyle='-.', color='royalblue')
    #ax.plot(np.array((0, 0)), np.array((-border, border)), linestyle='-.', color='royalblue')


In [416]:
def gaussian_optimizer(start_x, mu=None, sig=None, A=-1):
    mu = np.zeros(2) if mu is None else mu
    sig = np.ones(2) if sig is None else sig
    
    F = functools.partial(tc.gaussian, mu=mu, sig=sig, A=A)
    C = ()
    
    opt = GCobyla(start_x, F, C, rhobeg=0.5, rhoend=1e-12)
    return opt

In [418]:
plt.close('all')
fig, axes = plt.subplots(2, 3, figsize=(12, 9))
mu = (0, 0)

gg = functools.partial(gaussian_optimizer, mu=mu, A=-1)

rsc = {
    axes[0,0]: ((.5**.5,.5**.5),'a)'),
    axes[0,1]: ((0,1), 'b)'),
    axes[0,2]: ((-.5**.5,.5**.5), 'c)'),
    axes[1,0]: ((-.5**.5,-.5**.5), 'd)'),
    axes[1,1]: ((0,-1), 'e)'),
    axes[1,2]: ((.5**.5,-.5**.5),'f)'),
}
    

for ax, data in rsc.items():
    start_x, caption = data
    opt = gg(start_x=start_x)
    for _ in opt.g_run(): pass

    ax.set_xlabel(f'{caption} start x at ({start_x[0]:.2f}, {start_x[1]:.2f})')
    ax_plot_track(opt, ax, mu,  aspect=True, track_color='darkred')
    ax_plot_map(fig, ax, opt, border=2, nsamples=50)
    
plt.savefig('mem/rsc/ex1-1.png', dpi=120, bbox_inches='tight')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Una normal amb restriccions

In [420]:
def gaussian_optimizer(start_x, border=1, mu=None, sig=None, A=-1):
    mu = np.zeros(2) if mu is None else mu
    sig = np.ones(2) if sig is None else sig
    
    F = functools.partial(tc.gaussian, mu=mu, sig=sig, A=A)
    c1 = lambda x: border - x[0]
    c2 = lambda x: border + x[0]
    c3 = lambda x: border - x[1]
    c4 = lambda x: border + x[1]
    C = (c1, c2, c3, c4)
    
    opt = GCobyla(start_x, F, C, rhobeg=0.5, rhoend=1e-12)
    return opt

In [421]:
plt.close('all')
fig, axes = plt.subplots(2, 3, figsize=(12, 9))
border = .5
mu = (0, 0)

gg = functools.partial(gaussian_optimizer, border=border, mu=mu, A=-1)

rsc = {
    axes[0,0]: ((.5**.5,.5**.5),'a)'),
    axes[0,1]: ((0,1), 'b)'),
    axes[0,2]: ((-.5**.5,.5**.5), 'c)'),
    axes[1,0]: ((-.5**.5,-.5**.5), 'd)'),
    axes[1,1]: ((0,-1), 'e)'),
    axes[1,2]: ((.5**.5,-.5**.5),'f)'),
}
    

for ax, data in rsc.items():
    start_x, caption = data
    opt = gg(start_x=start_x)
    for _ in opt.g_run(): pass

    ax.set_xlabel(f'{caption} start x at ({start_x[0]:.2f}, {start_x[1]:.2f})')
    ax_plot_track(opt, ax, mu,  aspect=True, track_color='darkred')
    ax_plot_map(fig, ax, opt, border=2, nsamples=50)
    
    region = np.array(((border, border), (-border,border), (-border, -border), (border, -border), (border, border)))
    ax.plot(region[..., 0], region[..., 1], linestyle='-.', color='black')
    ax.fill(region[..., 0], region[..., 1], color='limegreen', alpha=.35)
    
plt.savefig('mem/rsc/ex1-2.png', dpi=120, bbox_inches='tight')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [423]:
plt.close('all')
fig, axes = plt.subplots(2, 3, figsize=(12, 9))
mu = (0, 0)
border = 1

gg = functools.partial(gaussian_optimizer, mu=mu, A=-1)

start_x = (.5**.5, .5**.5)
rsc = {
    axes[0,0]: (start_x, 'a)'),
    axes[0,1]: (start_x, 'b)'),
    axes[0,2]: (start_x, 'c)'),
    axes[1,0]: (start_x, 'd)'),
    axes[1,1]: (start_x, 'e)'),
    axes[1,2]: (start_x, 'f)'),
}
    

for ax, data in rsc.items():
    start_x, caption = data
    opt = gg(start_x=start_x, border=border)
    for _ in opt.g_run(): pass

    ax.set_xlabel(f'{caption} start x at ({start_x[0]:.2f}, {start_x[1]:.2f}), length side {2*border:.2f}')
    ax_plot_track(opt, ax, mu,  aspect=True, track_color='darkred')
    ax_plot_map(fig, ax, opt, border=2, nsamples=50)
    
    region = np.array(((border, border), (-border,border), (-border, -border), (border, -border), (border, border)))
    ax.plot(region[..., 0], region[..., 1], linestyle='-.', color='black')
    ax.fill(region[..., 0], region[..., 1], color='limegreen', alpha=.35)
    border *= .5
plt.savefig('mem/rsc/ex1-3.png', dpi=120, bbox_inches='tight')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Normal amb regió delimitada per dues rectes. x inicial fora de la regió especial

In [424]:
def gaussian_optimizer(start_x, mu=None, sig=None, A=-1):
    mu = np.zeros(2) if mu is None else mu
    sig = np.ones(2) if sig is None else sig
    
    F = functools.partial(tc.gaussian, mu=mu, sig=sig, A=A)
    c1 = lambda x: x[0] - x[1] - .5
    c2 = lambda x: -(x[0] - x[1] - 1)
    C = (c1, c2)
    
    opt = GCobyla(start_x, F, C, rhobeg=0.5, rhoend=1e-12)
    return opt

In [425]:
plt.close('all')
fig, axes = plt.subplots(1,2, figsize=(9, 4))
mu = (0, 0)

gg = functools.partial(gaussian_optimizer, mu=mu, A=-1)

rsc = {
    axes[0]: ((-1, 0), 'a)'),
    axes[1]: ((1.8, 0), 'b)'),
}
    
for ax, data in rsc.items():
    start_x, caption = data
    opt = gg(start_x=start_x)
    for _ in opt.g_run(): pass

    ax.set_xlabel(f'{caption} start x at ({start_x[0]:.2f}, {start_x[1]:.2f})')
    ax_plot_track(opt, ax, mu,  aspect=True, track_color='darkred')
    ax_plot_map(fig, ax, opt, border=2, nsamples=100)
    
    l1 = lambda x: x - .5
    l2 = lambda x: x - 1
    c1 = np.array(((-1.5, l1(-1.5)), (2, l1(2))))
    c2 = np.array(((-1, l2(-1)), (2, l2(2))))
    ax.plot(c1[..., 0], c1[..., 1], linestyle='-.', color='black')
    ax.plot(c2[..., 0], c2[..., 1], linestyle='-.', color='black')
    
    region = np.array(((-1.5, l1(-1.5)), (2, l1(2)), (2, l2(2)), (-1, l2(-1)), (-1.5, l1(-1.5))))
    ax.fill(region[..., 0], region[..., 1], color='limegreen', alpha=.35)
    print(opt.optimal_vertex)

plt.savefig('mem/rsc/ex1-4.png', dpi=120, bbox_inches='tight')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[ 0.25 -0.25]
[ 0.25 -0.25]


In [426]:
import matplotlib.cm as cm

plt.close('all')
fig, ax = plt.subplots(1, figsize=(9, 4))
mu = (0, 0)

gg = functools.partial(gaussian_optimizer, mu=mu, A=-1)
angles = np.arange(start=0, stop=2*np.pi, step=15*(2*np.pi)/360)

for angle in angles:
    start_x = np.array((np.cos(angle), np.sin(angle)))
    opt = gg(start_x=start_x)
    for _ in opt.g_run(): pass

    ax_plot_track(opt, ax, mu,  aspect=True, track_color=None)
    ax_plot_map(fig, ax, opt, border=2, nsamples=100)
    
    l1 = lambda x: x - .5
    l2 = lambda x: x - 1
    c1 = np.array(((-1.5, l1(-1.5)), (2, l1(2))))
    c2 = np.array(((-1, l2(-1)), (2, l2(2))))
    ax.plot(c1[..., 0], c1[..., 1], linestyle='-.', color='black')
    ax.plot(c2[..., 0], c2[..., 1], linestyle='-.', color='black')

    assert (opt.optimal_vertex.round(9) == np.array((.25, -.25))).all()
    #print(opt.optimal_vertex)
region = np.array(((-1.5, l1(-1.5)), (2, l1(2)), (2, l2(2)), (-1, l2(-1)), (-1.5, l1(-1.5))))
ax.fill(region[..., 0], region[..., 1], color='limegreen', alpha=.35)
plt.savefig('mem/rsc/ex1-5.png', dpi=120, bbox_inches='tight')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Out of circle

In [454]:
def optimization_out_of_the_boundary_circle(inner_radius=1, outer_radius=2, start_x=(0,0)):
    assert outer_radius > inner_radius
    
    sig = np.random.choice((-1, 1))
    phi = np.random.uniform(low=0, high=2*np.pi*outer_radius) / outer_radius
    x_circle = outer_radius * np.cos(phi)
    y_circle = outer_radius * np.sin(phi)
    
    mu = np.array((x_circle, y_circle))
    
    G = functools.partial(tc.gaussian, mu=mu, A=-1)
    c1 = lambda x: inner_radius - sum(x ** 2)

    C = (c1,)

    opt = GCobyla(start_x, G, C, rhobeg=3e-1, rhoend=1e-15)
    for _ in opt.g_run(): pass
    
    
    return mu, opt

In [460]:
%matplotlib widget
import matplotlib.pyplot as plt
plt.close('all')

#start_x = np.random.uniform(low=-1, high=1, size=2)
start_x = np.array((-0.10696211, -0.65293934))
inner_radius = 1
outer_radius = 2
mu, opt = optimization_out_of_the_boundary_circle(inner_radius=inner_radius, outer_radius=outer_radius, start_x=start_x)

plt.close('all')
fig, ax = plt.subplots(figsize=(6, 6))
ax.axis('equal')

inner_circle = plt.Circle((0, 0), inner_radius, color='limegreen', fill=True, alpha=.35)  # boundary
outer_circle = plt.Circle((0, 0), outer_radius, color='lightgrey', fill=False)
ax.add_patch(inner_circle)
ax.add_patch(outer_circle)

#ax.plot(*start_x, marker='o', color='blue')
#ax.plot(*opt.x, marker='o', color='orange')
ax_plot_track(opt, ax, mu,  aspect=True, track_color='darkred')
#ax.plot(*mu, marker='*', color='limegreen')
#ax.plot((opt.x[0], mu[0]), (opt.x[1], mu[1]), linestyle='-.', color='darkred')
plt.savefig('mem/rsc/ex1-6.png', dpi=120, bbox_inches='tight')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [497]:
print(f"{'x_opt':>5}: {opt.optimal_vertex}")
print(f"{'mu':>5}: {mu}")
dist = np.linalg.norm(opt.optimal_vertex - .5*mu)
print(f"{'dist':>5}: {dist}")
assert dist < 1e-10

x_opt: [-0.12998324  0.99151619]
   mu: [-0.25996647  1.98303238]
 dist: 5.877132908087334e-11


In [488]:
print(f'{np.array((1,2))!s:>20}')

       array([1, 2])
